In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import os

In [ ]:
b30 = []
blist = []
h30 = []
hlist = []

In [ ]:
pth1 = '/kaggle/input/gearbox-fault-diagnosis/BrokenTooth'
pth2 = '/kaggle/input/gearbox-fault-diagnosis/Healthy'

In [ ]:
for file in os.listdir(pth1):
    blist.append(os.path.join(pth1, file))
blist.sort()
blist

In [ ]:
for file in os.listdir(pth2):
    hlist.append(os.path.join(pth2, file))
hlist.sort()
hlist

INPUT Number of rows for aggregated dataset

In [ ]:
grouped = 100

## Read datasets
### Healthy gearbox

In [ ]:
for file in blist:
    b30.append(pd.read_csv(file))

In [ ]:
for file in hlist:
    h30.append(pd.read_csv(file))

In [ ]:
b30[8]

## Compute aggregated datasets
### Function to compute aggregated features: standard deviation

In [ ]:
# This function computes the standard deviation for a sample, where:
##  - df:        the dataset, one variable per column
##  - grouped:   the number of consecutive points (rows) in the time series to compute the standard deviation
##  - outcome:   column added to the dataset representing healthy(=1) or failure (=0)
##  RETURNS a dataset (df_result) where each column is the time serie of every variable

def stdev_features(df, grouped, load, outcome):
    #### Create empty dataframe with columns a1,a2,a3,a4
    df_result = pd.DataFrame( [ np.zeros(len(df.columns)-2) ],columns= df.columns[:-2])

    #### Aggregate in groups of nrows computing the standard deviation
    # Remove load & failure columns, keeping only a1,a2,a3,a4
    stdev_lenght=len(df.columns)-2
    
    #### Compute number of rows of the aggregated dataframe
    nrows_raw = len(df.index)
    nrows = math.floor(nrows_raw / grouped)
    nrows_dropped = nrows_raw - nrows*grouped
    print("nrows_raw=", nrows_raw, "   nrows=", nrows, "   Number of dropped rows of grouped= ", nrows_dropped/grouped*100,"%\n")
    
    # Iterate every 'grouped' rows and compute stdev per column
    for i in range(nrows):
        df_group = df.iloc[i*grouped:i*grouped+grouped,:]
        df_stdev = pd.DataFrame(df_group.std()).transpose()
        # Remove load & failure columns
        df_stdev=df_stdev.iloc[:,:stdev_lenght]
        # Add row of calculated stdev
        df_result = df_result.append(df_stdev, ignore_index=True)

        #print ("i*grouped TO i*grouped+grouped", i*grouped, i*grouped+grouped)
        #print ("row, df_stdev=\n", row, df_stdev.iloc[:,:])
        #print ("df_result=\n", df_result)
        #print("row", i, "\n", df_group)


    # Remove the first row (it was the seed of zeros for initializing df_result)
    #print(df_result)
    df_result = df_result.iloc[1:,:]
    # Add the column for 'load'
    df_result['load'] = load*np.ones((nrows,1))

    # Add the column for 'failure'
    failure = np.ones((nrows,1))
    df_result['failure'] = outcome
    print(df_result)
    
    return df_result

### Load= 0%

In [ ]:
b30[0].describe()

In [ ]:
b30_stdev = []
h30_stdev = []

In [ ]:
list(range(len(b30)))

In [ ]:
for i in range(len(b30)):
    load = 10*i

    b30[i]["load"] = load

    ##   0 is healthy
    ##   1 is broken
    failure = 1
    b30[i]['failure'] = failure
    
    b30_stdev.append(stdev_features (df = b30[i], grouped = grouped, load = load, outcome = failure))

In [ ]:
for i in range(len(h30)):
    load = 10*i

    h30[i]["load"] = load

    ##   0 is healthy
    ##   1 is broken
    failure = 0
    h30[i]['failure'] = failure
    
    h30_stdev.append(stdev_features (df = h30[i], grouped = grouped, load = load, outcome = failure))

In [ ]:
len(b30_stdev)

## Compare stdev of acceleration traversing % load

In [ ]:
print("Load 0%")
print("-------")
stdev_bhz0 = pd.DataFrame(b30_stdev[0].iloc[:,:-2].mean()).transpose()
stdev_hhz0 = pd.DataFrame(h30_stdev[0].iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz0)
print("Healthy:\n", stdev_hhz0)

In [ ]:
print("Load 10%")
print("--------")
stdev_bhz10 = pd.DataFrame(b30_stdev[1].iloc[:,:-2].mean()).transpose()
stdev_hhz10 = pd.DataFrame(h30_stdev[1].iloc[:,:-2].mean()).transpose()
print("Broken:\n", stdev_bhz10)
print("Healthy:\n", stdev_hhz10)

## Stacked BROKEN dataframe

In [ ]:
b30hz_stdev = b30_stdev[0]

In [ ]:
for i in range(1,10):
    b30hz_stdev = b30hz_stdev.append(b30_stdev[i])

In [ ]:
b30hz_stdev.describe()

## Export BROKEN dataframe as CSV

In [ ]:
b30hz_stdev.to_csv('/kaggle/working/b30hz_stdev_100.csv', index = False)

## Stacked HEALTHY dataframe

In [ ]:
h30hz_stdev = h30_stdev[0]

In [ ]:
for i in range(1,10):
    h30hz_stdev = h30hz_stdev.append(h30_stdev[i])

In [ ]:
h30hz_stdev.describe()

## Export HEALTHY dataframe as CSV

In [ ]:
h30hz_stdev.to_csv('/kaggle/working/h30hz_stdev_100.csv', index = False)